In [3]:
from seleniumbase import Driver
from selenium.webdriver.common.by import By
import pandas as pd
import time


In [13]:
driver = Driver(uc=True)   # undetected-chromedriver

In [5]:
BASE_URL = 'https://www.noon.com/egypt-ar/search/?page={}&q=%D8%AD%D9%88%D8%B6%20%D8%A7%D8%B3%D8%AA%D8%A7%D9%86%D9%84%D8%B3'

In [6]:
all_product_links = []
driver.implicitly_wait(10)

In [7]:
for i in range(1,20):
    driver.get(BASE_URL.format(i))
    time.sleep(3) 
    PRODUCT_NAME_LOCATOR = (By.XPATH, '//h2[@data-qa="plp-product-box-name"]')
    product_name_elements = driver.find_elements(*PRODUCT_NAME_LOCATOR)
    
    if not product_name_elements:
        print("No product elements found on the page.")
    
    for index, name_element in enumerate(product_name_elements):
        link_element = name_element.find_element(By.XPATH, './ancestor::a[1]')
        product_link = link_element.get_attribute('href')
        all_product_links.append(product_link)


In [8]:
all_product_links

['https://www.noon.com/egypt-ar/2-piece-stainless-steel-faucet-connector-flexible-movable/ZB88924D7826D34F42ADCZ/p/?o=c66dffb265686fdc',
 'https://www.noon.com/egypt-ar/stainless-steel-kitchen-sink-80-x-48-cm/Z77EC0860032CE1693337Z/p/?o=z77ec0860032ce1693337z-1',
 'https://www.noon.com/egypt-ar/kitchen-sink-stainless-steel-62-45-1mm-kitchen-sink-62-cm-cup-washer-3-inch-drainer/Z580EFBE72CF2DD7FD2B1Z/p/?o=z580efbe72cf2dd7fd2b1z-1',
 'https://www.noon.com/egypt-ar/stainless-steel-kitchen-sink-50-x-80-sink-drainer-black-pull-out-mixer-collapsible-silicone-dish-and-drainer/Z004C26D90EA637C79D2EZ/p/?o=z004c26d90ea637c79d2ez-1',
 'https://www.noon.com/egypt-ar/2-stainless-steel-anti-rust-strainers-stainless-steel-strainer-holder-and-spoon-holder-for-sink-and-dining-room-equipped-with-a-jumbo-stainless-steel-strainer/Z0067E65CA7A587DB84EEZ/p/?o=z0067e65ca7a587db84eez-1',
 'https://www.noon.com/egypt-ar/1st-stainless-75cm-by-45cm-thick-1mm/Z89688B2C74B065FF5A67Z/p/?o=z89688b2c74b065ff5a67z-1',

In [11]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

def extract_noon_reviews(driver, url, timeout=15):
    """
    Extract all reviews from a Noon product page.
    
    Args:
        driver: Selenium WebDriver
        url: Product URL
        timeout: Max seconds to wait for reviews to load

    Returns:
        List of dictionaries with review info
    """
    driver.get(url)
    all_reviews = []
    wait = WebDriverWait(driver, timeout)
    product_name = driver.find_element(By.CSS_SELECTOR, "[class*='ProductTitle']").text

    # ---- Gradual scroll until reviews section is visible ----
    def scroll_to_reviews():
        for _ in range(5):
            try:
                review_section = driver.find_element(By.XPATH, "//div[contains(@class,'ProductReviewsFilters-module') and contains(., 'المراجعات')]")
                if review_section.is_displayed():
                    driver.execute_script("arguments[0].scrollIntoView(true);", review_section)
                    return True
            except:
                pass
            driver.execute_script("window.scrollBy(0, 550);")
            time.sleep(0.4)
        return False

    if not scroll_to_reviews():
        print("⚠️ Reviews section not found on page")
        return []

    time.sleep(1)  # small wait for rendering

    # ---- Click "View All Reviews" if exists ----
    try:
        view_all = driver.find_element(By.CSS_SELECTOR, ".ViewAllReviewsBtn-module-scss-module__viewMore")
        driver.execute_script("arguments[0].scrollIntoView(true);", view_all)
        time.sleep(0.5)
        view_all.click()
        time.sleep(1.1)  # wait for all reviews page to load
    except NoSuchElementException:
        pass  # no "View All Reviews" button

    # ---- Wait until at least one review is visible or timeout ----
    try:
        wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, ".ReviewListSection-module-scss-module__JSH-ya__noonReviewItem")
            )
        )
    except TimeoutException:
        print("⚠️ No reviews found for this product")
        return []

    # ---- Extract reviews ----
    review_containers = driver.find_elements(
        By.CSS_SELECTOR, ".ReviewListSection-module-scss-module__JSH-ya__noonReviewItem"
    )

    for r in review_containers:
        try:
            username = r.find_element(By.CSS_SELECTOR, ".ReviewItemHeader-module-scss-module__W2V5NW__userName").text
        except:
            username = "anonymous"

        try:
            date = r.find_element(By.CSS_SELECTOR, ".ReviewItemHeader-module-scss-module__W2V5NW__ratedDate").text
        except:
            date = ""

        try:
            title = r.find_element(By.CSS_SELECTOR, ".TranslatedReviewTitle-module-scss-module__QbaJfW__reviewTitle").text
        except:
            title = ""

        try:
            body = r.find_element(By.CSS_SELECTOR, ".ReviewDescription-module-scss-module__AkZflW__reviewDesc").text
        except:
            body = ""

        try:
            rating_imgs = r.find_elements(By.CSS_SELECTOR, ".RatingStars-module-scss-module__fF13gq__container img")
            rating = sum(1 for img in rating_imgs if "rating" in img.get_attribute("color") and "grey" not in img.get_attribute("color"))
        except:
            rating = None

        all_reviews.append({
            "product": product_name,
            "username": username,
            "date": date,
            "title": title,
            "text": body,
            "rating": rating
        })

    print(f"✅ Found {len(all_reviews)} review(s)")
    return all_reviews


In [14]:
all_reviews = []
for url in all_product_links:
    reviews = extract_noon_reviews(driver,url)
    all_reviews.extend(reviews)  # Combine all reviews in one list


⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
✅ Found 1 review(s)
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
✅ Found 2 review(s)
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on pag

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[class*='ProductTitle']"}
  (Session info: chrome=139.0.7258.154); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
#0 0x615e8232908a <unknown>
#1 0x615e81dc8a70 <unknown>
#2 0x615e81e1a907 <unknown>
#3 0x615e81e1ab01 <unknown>
#4 0x615e81e68d54 <unknown>
#5 0x615e81e4040d <unknown>
#6 0x615e81e6614f <unknown>
#7 0x615e81e401b3 <unknown>
#8 0x615e81e0c59b <unknown>
#9 0x615e81e0d971 <unknown>
#10 0x615e822ee25b <unknown>
#11 0x615e822f1fa9 <unknown>
#12 0x615e822d5339 <unknown>
#13 0x615e822f2b58 <unknown>
#14 0x615e822b9c1f <unknown>
#15 0x615e82316118 <unknown>
#16 0x615e823162f6 <unknown>
#17 0x615e82328066 <unknown>
#18 0x7243e4e9caa4 <unknown>
#19 0x7243e4f29c6c <unknown>


In [15]:
df_all_reviews = pd.DataFrame(all_reviews)
df_all_reviews.head()

,product,username,date,title,text,rating
0,حوض استانلس الأول (75 سم في 45 سم سمك 1 مم بال...,ابراهيم ❤️❤️ ح.,"أكتوبر 31, 2024",جميل,جاري الاستخدام,5
1,مصفاة حوض من الستانلس ستيل للمطبخ والحمام فضي ...,اميره م.,"سبتمبر 9, 2025",حلو اووووووي,كبير و بيغطي العين كلها و حلو بصراحه ممتاز متو...,5
2,مصفاة حوض من الستانلس ستيل للمطبخ والحمام فضي ...,Midhun P.,"يناير 14, 2025",Good,Good,5
3,شماعة للباب من الستانلس ستيل فضي,Mohamed A.,"سبتمبر 21, 2024",جودة متوسطة,لاء يتناسب للتعليق علي جميع الابواب,3
4,شماعة للباب من الستانلس ستيل فضي,Anonymous,"أكتوبر 27, 2023",زى الصوره,الباب مابيقفلش لما الشماعه بتركب,3


In [16]:
df_all_reviews.duplicated().sum()

np.int64(1)

In [17]:
df_all_reviews.drop_duplicates(inplace=True)

In [18]:
df_all_reviews.shape

(51, 6)

In [19]:
df_all_reviews.to_csv("all_noon_review_sink.csv", index=False)